In [1]:

import plotly.express as px

import dash
from dash import dcc
from dash import html

import plotly.graph_objects as go


from dash import Dash, Input, Output, State, callback

from jupyter_dash import JupyterDash

import dash_bootstrap_components as dbc
COMPONENT_STYLE = "/assets/my_component.css"
external_stylesheets=[dbc.themes.BOOTSTRAP]

app = JupyterDash(__name__, external_stylesheets=external_stylesheets,
                  meta_tags=[{'name': 'viewport', 'content': 'width=device-width, initial-scale=1'}],
                 suppress_callback_exceptions=True)
# Create server variable with Flask server object for use with gunicorn
server = app.server

In [2]:

#from dash.dependencies import Input, Output, State
import copy
import json

In [3]:
FIGURE = {
    'data': [{
        'x': [1, 2, 3],
        'y': [4, 3, 6],
        'mode': 'markers',
        'marker': {
            'size': 8
        }
    }],
    'layout': {
        'xaxis': {},
        'yaxis': {},
    }
}

app.layout = html.Div([
    html.H3('Persistent Zoom on Updates'),
    html.Div('''
        Try zooming into the graph (clicking and dragging on the graph),
        then updating the text box. Notice how the graph does not zoom out
        when the graph is updated. Switching to "Refresh View" will
        redraw the graph with auto-range enabled.
    '''),
    dcc.Input(id='my-input'),
    dcc.RadioItems(
        id='lock-zoom',
        options=[{'label': i, 'value': i} for i in ['Lock View', 'Refresh View']],
        value='Lock View'
    ),
    dcc.Graph(
        id='my-graph',
        figure=FIGURE
    ),
     html.Div(id='layout-data'),
])

@app.callback(
    [Output('my-graph', 'figure'),Output(component_id='layout-data', component_property='children')],
    [Input('my-input', 'value'),
     Input('lock-zoom', 'value')],
    [State('my-graph', 'relayoutData')])
def update_graph(value, lock_zoom, relayout_data):
    new_figure = copy.deepcopy(FIGURE)
    new_figure['layout']['title'] = value

    # relayout_data contains data on the zoom and range actions
    # print(relayout_data)
    #result = relayout_data.items()
    result = json.dumps(relayout_data)
    if relayout_data and lock_zoom == 'Lock View':
        if 'xaxis.range[0]' in relayout_data:
            new_figure['layout']['xaxis']['range'] = [
                relayout_data['xaxis.range[0]'],
                relayout_data['xaxis.range[1]']
            ]
        if 'yaxis.range[0]' in relayout_data:
            new_figure['layout']['yaxis']['range'] = [
                relayout_data['yaxis.range[0]'],
                relayout_data['yaxis.range[1]']
            ]

    return new_figure , result

#@app.callback(
#    Input('my-input', 'value'),
#    Output(component_id='layout-data', component_property='children'))
#def update_output_div(relayout_data):
#    #result = json.dumps(relayout_data)
#    result = relayout_data.items()
#    return result

In [4]:
##app.layout = layout3
app.run_server(debug=True,port=5051)

Dash app running on http://127.0.0.1:5051/
